# Kernel Density Estimation

Kernel Density Estimate (KDE) is a statistical technique to estimate the probability density function of a random variable.
In this notebook, we employ KDE to visualize the distribution of tweets that contain a certan term over time.

## Import Dependencies

In [ ]:
%AddDeps com.lucidworks.spark spark-solr 3.6.0 --transitive

## Query Solr

In [1]:
def updateTime(hour:Int, createdAt:String):Int = {
    var adjusted = hour
    createdAt match {
      case "Pacific Time (US & Canada)" => adjusted = shiftHours(hour, -8)
      case "Eastern Time (US & Canada)" => adjusted = shiftHours(hour, -5)
      case "Central Time (US & Canada)" => adjusted = shiftHours(hour, -5)
      case "Mountain Time (US & Canada)" => adjusted = shiftHours(hour, -6)
      case "Atlantic Time (Canada)" => adjusted = shiftHours(hour, -4)
    }
    adjusted
}

def timeZoneToInt(timeZone:String):Int = {
    var out = 6 // sunday

    if (timeZone contains "Mon") {
      out = 0
    } else if (timeZone contains "Tue") {
      out = 1
    } else if (timeZone contains "Wed") {
      out = 2
    } else if (timeZone contains "Thu") {
      out = 3
    } else if (timeZone contains "Fri") {
      out = 4
    } else if (timeZone contains "Sat") {
      out = 5
    }
    out
}

def shiftHours(hour:Int, shift:Int):Int = {
    var adjusted = hour + shift
    if (adjusted >= 24) {
      adjusted %= 24
    } else if (adjusted < 0) {
      adjusted += 24
    }
    adjusted
}

updateTime: (hour: Int, createdAt: String)Int
timeZoneToInt: (timeZone: String)Int
shiftHours: (hour: Int, shift: Int)Int


First we find the tweets that contain the term TERM, which are created in Canada or USA. We accumulate the tweets over a certain time period, MODE (e.g: day or hour).

In [24]:
import com.lucidworks.spark.rdd.SelectSolrRDD
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.mllib.stat.KernelDensity
import org.apache.hadoop.fs.{FileSystem, Path}
import play.api.libs.json._

// Solr's ZooKeeper URL
val SOLR = "192.168.1.111:9983"

// The Solr collection
val INDEX = "mb13"

// The Solr query
val MODE = "day"  // day OR hour
val TERM = "school"
val QUERY = s"contents:${TERM}"

// The limit for number of rows to process
val LIMIT = 1000

// Output directory
val OUT_DIR = "kde"

// Delete old output dir
FileSystem.get(sc.hadoopConfiguration).delete(new Path(OUT_DIR), true)

val timeRegex = raw"([0-9]+):([0-9]+):([0-9]+)".r

val rdd = new SelectSolrRDD(SOLR, INDEX, sc, maxRows = Some(LIMIT))
.rows(1000)
.query(QUERY)
.flatMap(doc => {
    val parsedJson = Json.parse(doc.get("raw").toString)
    var out:List[Tuple3[Int, Double, Int]] = List()

    try {
        val timeZone:String = (parsedJson \ "user" \ "time_zone").as[String]
        if ((timeZone contains "Canada") || (timeZone contains "US")) {
            val time = (parsedJson \ "created_at").as[String]
            val matches = timeRegex.findFirstMatchIn(time)
            val hour = updateTime(matches.get.group(1).toInt, timeZone)
            val week = timeZoneToInt(time)
            val min = matches.get.group(2).toDouble
            out = if (MODE == "day") List((week, hour/24, 1)) else List((hour, min/60, 1))        
            }
        } catch {
          case e : Exception => println("unable to parse the tweet", e)
        }
        out
      }).persist()

SOLR = 192.168.1.111:9983
INDEX = mb13
MODE = day
TERM = school
QUERY = contents:school
LIMIT = 1000
OUT_DIR = kde
timeRegex = ([0-9]+):([0-9]+):([0-9]+)
rdd = MapPartitionsRDD[146] at flatMap at <console>:220


MapPartitionsRDD[146] at flatMap at <console>:220

## Compute KDE

In [25]:
val counts = rdd.map(item => (item._1, item._3)).reduceByKey(_+_).sortByKey().collect().toMap

val kdeData = rdd.map(item => item._1.toInt.toDouble + item._2)

val kd = if (MODE == "day") new KernelDensity().setSample(kdeData).setBandwidth(1.0) else new KernelDensity().setSample(kdeData).setBandwidth(2.0)
val domain = if (MODE ==  "day") (0 to 6).toArray else (0 to 23).toArray

val densities = kd.estimate(domain.map(_.toDouble))

println(s"counts / density per $MODE for $TERM")
domain.foreach(x => {
    println(s"$x ( ${counts(x)} ) -- ${densities(x)}")
})

sc.parallelize(densities).coalesce(1).saveAsTextFile(OUT_DIR)

counts / density per day for school
0 ( 82 ) -- 0.14522323398019227
1 ( 79 ) -- 0.18434542720787753
2 ( 69 ) -- 0.17010762720068012
3 ( 46 ) -- 0.1419207838156945
4 ( 56 ) -- 0.11539841926988921
5 ( 21 ) -- 0.08393051411578506
6 ( 31 ) -- 0.053867987743947854


counts = Map(0 -> 82, 5 -> 21, 1 -> 79, 6 -> 31, 2 -> 69, 3 -> 46, 4 -> 56)
kdeData = MapPartitionsRDD[150] at map at <console>:197
kd = org.apache.spark.mllib.stat.KernelDensity@378acfaf
domain = Array(0, 1, 2, 3, 4, 5, 6)
densities = Array(0.14522323398019227, 0.18434542720787753, 0.17010762720068012, 0.1419207838156945, 0.11539841926988921, 0.08393051411578506, 0.053867987743947854)


Array(0.14522323398019227, 0.18434542720787753, 0.17010762720068012, 0.1419207838156945, 0.11539841926988921, 0.08393051411578506, 0.053867987743947854)

## Generate Graph

In [26]:
import sys.process._

// Remove the old output directory
"rm -rf kde.png /tmp/kde" !

// Copy new output from HDFS to local filesystem
"hdfs dfs -copyToLocal kde /tmp/kde" !

// Generate the word cloud
if (MODE == "day") {
    "python kde_day.py" !
} else {
    "python kde_hour.py" !
}

No handles with labels found to put in legend.
No handles with labels found to put in legend.


0

![](kde.png)